In [40]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, LSTM
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping

In [42]:


# # Define CNN model
# def create_cnn_model(input_shape, num_classes):
#     model = Sequential()
#     model.add(Dense(32, activation='relu', input_shape=input_shape))
#     model.add(MaxPooling1D(pool_size=3))
#     # model.add(Conv1D(32, kernel_size=3, activation='relu'))
#     # model.add(MaxPooling1D(pool_size=2))
#     model.add(LSTM(64))
#     model.add(Flatten())
#     model.add(Dense(64, activation='relu'))
#     model.add(Dense(num_classes, activation='softmax'))
#     model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#     return model
# def create_cnn_model(input_shape, num_classes):
#     model = Sequential()
#     model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape))
#     model.add(MaxPooling1D(pool_size=2))
#     model.add(Conv1D(32, kernel_size=3, activation='relu'))
#     model.add(MaxPooling1D(pool_size=2))
#     model.add(LSTM(64))  # Adding LSTM layer
#     model.add(Flatten())
#     model.add(Dense(64, activation='relu'))
#     model.add(Dense(num_classes, activation='softmax'))
#     model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#     return model


def create_nn_model(input_shape, num_classes):
    model = Sequential([
        Dense(128, activation='relu', input_shape=input_shape),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Flatten(),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [39]:
X = pd.read_csv(r'E:\4th sem\IOBS-2\project\Non Coding RNA classification\Cleaned_data\EIIP\0-100_features.csv')
y = pd.read_csv(r"E:\4th sem\IOBS-2\project\Non Coding RNA classification\Cleaned_data\EIIP\0-100_class.csv")

In [5]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and transform labels
encoded_labels = label_encoder.fit_transform(y)

c:\Users\neelr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
print(len(X),len(y))

1452 1452


In [36]:
from sklearn.model_selection import train_test_split

# Split data into train (80%) and combined test/validation (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, encoded_labels, test_size=0.1, shuffle=True, random_state=42)

# Split combined test/validation into test (10%) and validation (10%)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.50, shuffle=True, random_state=42)

# Display the shapes of the resulting sets
print("Train set shapes:", X_train.shape, y_train.shape)
print("Test set shapes:", X_test.shape, y_test.shape)
print("Validation set shapes:", X_val.shape, y_val.shape)


Train set shapes: (1306, 100) (1306,)
Test set shapes: (73, 100) (73,)
Validation set shapes: (73, 100) (73,)


In [43]:
# Preprocess data
max_sequence_length = 100  # Example maximum sequence length


# Reshape data for CNN input
input_shape = (X_train.shape[1], 1)

# Number of classes
num_classes = len(np.unique(y_train))

# Create CNN model
model = create_nn_model(input_shape, num_classes)

model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 100, 128)          256       
                                                                 
 dropout (Dropout)           (None, 100, 128)          0         
                                                                 
 dense_31 (Dense)            (None, 100, 64)           8256      
                                                                 
 dropout_1 (Dropout)         (None, 100, 64)           0         
                                                                 
 flatten_11 (Flatten)        (None, 6400)              0         
                                                                 
 dense_32 (Dense)            (None, 12)                76812     
                                                                 
Total params: 85324 (333.30 KB)
Trainable params: 853

In [44]:

# Train CNN model
# model.fit(X, encoded_labels, epochs=200)
# Train the model
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_val, y_val))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Print test accuracy
print("Test Accuracy:", test_accuracy)


Epoch 1/30
41/41 [==============================] - 2s 21ms/step - loss: 1.9461 - accuracy: 0.3469 - val_loss: 1.8357 - val_accuracy: 0.4247
Epoch 2/30
41/41 [==============================] - 1s 12ms/step - loss: 1.5106 - accuracy: 0.5360 - val_loss: 1.3313 - val_accuracy: 0.5890
Epoch 3/30
41/41 [==============================] - 0s 12ms/step - loss: 1.0413 - accuracy: 0.6608 - val_loss: 1.0662 - val_accuracy: 0.6438
Epoch 4/30
41/41 [==============================] - 1s 14ms/step - loss: 0.8122 - accuracy: 0.7221 - val_loss: 1.0270 - val_accuracy: 0.6438
Epoch 5/30
41/41 [==============================] - 1s 12ms/step - loss: 0.7142 - accuracy: 0.7588 - val_loss: 1.0571 - val_accuracy: 0.6575
Epoch 6/30
41/41 [==============================] - 0s 12ms/step - loss: 0.6518 - accuracy: 0.7779 - val_loss: 1.0048 - val_accuracy: 0.6712
Epoch 7/30
41/41 [==============================] - 1s 13ms/step - loss: 0.6163 - accuracy: 0.7864 - val_loss: 1.0185 - val_accuracy: 0.6438
Epoch 8/30
41

KeyboardInterrupt: 

In [ ]:
X_200 = pd.read_csv(r'E:\4th sem\IOBS-2\project\Non Coding RNA classification\Cleaned_data\100-200_features.csv')
y_200 = pd.read_csv(r"E:\4th sem\IOBS-2\project\Non Coding RNA classification\Cleaned_data\100-200_class.csv")

from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and transform labels
encoded_labels_200 = label_encoder.fit_transform(y_200)

c:\Users\neelr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
encoded_labels_200

In [ ]:

# Now, you want to train the model for a new input length (200) without changing the architecture
# Assuming you have new data X_200 and encoded_labels_200 for the new input length

# Create a new CNN model with the same architecture as the previous model
# No need to compile the model as we will be loading the weights
model = create_cnn_model((200, 1), 13)

# Load weights from the previously trained model
# new_model.set_weights(model.get_weights())

# Train the new model with the new data for 200 input length
model.fit(X_200, encoded_labels_200, epochs=30)



Epoch 1/30
94/94 [==============================] - 2s 7ms/step - loss: 2.0225 - accuracy: 0.3435
Epoch 2/30
94/94 [==============================] - 1s 7ms/step - loss: 1.5691 - accuracy: 0.4851
Epoch 3/30
94/94 [==============================] - 1s 6ms/step - loss: 1.3541 - accuracy: 0.5733
Epoch 4/30
94/94 [==============================] - 1s 6ms/step - loss: 1.1960 - accuracy: 0.6215
Epoch 5/30
94/94 [==============================] - 1s 6ms/step - loss: 1.0711 - accuracy: 0.6579
Epoch 6/30
94/94 [==============================] - 1s 6ms/step - loss: 0.9670 - accuracy: 0.6943
Epoch 7/30
94/94 [==============================] - 1s 6ms/step - loss: 0.8875 - accuracy: 0.7207
Epoch 8/30
94/94 [==============================] - 1s 6ms/step - loss: 0.8085 - accuracy: 0.7421
Epoch 9/30
94/94 [==============================] - 1s 6ms/step - loss: 0.7309 - accuracy: 0.7655
Epoch 10/30
94/94 [==============================] - 1s 6ms/step - loss: 0.6782 - accuracy: 0.7868
Epoch 11/30
94/94 [